## Multiple Chain 快速入门

Runnables 可以轻松地用来串联多个 Chains，使用 RunnablePassthrough 将输出同时传给多条后继链。

```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

本指南展示如何使用 Runnable 实现多个 AI 关于相同话题的辩论：

```
    输入话题
       |
       |
    原始观点
      / \
     /   \
 正面论述 反面论述
     \   /
      \ /
     最终总结
```

In [3]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 创建正面论证的处理链，列出关于基础回应的正面或有利的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的正面或有利的方面"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的反面或不利的方面"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出批评后生成最终回应"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [4]:
chain.invoke({"input": "房地产低迷"})

'在面对房地产低迷的情况下，我们需要综合考虑正面和反面观点，以找到平衡和解决问题的方法。尽管房地产市场低迷可能会带来一些负面影响，但也有一些积极的方面。我们可以通过政府政策的引导和市场自我调整，来应对潜在的风险和挑战。同时，房地产市场的低迷也可能为购房者提供更好的购房机会，促进市场的健康发展。因此，我们应该以开放的心态和理性的态度来看待房地产市场的变化，并寻求合适的解决方案，以实现经济的平稳发展和社会的和谐进步。'

#### 流式输出

In [6]:
## chain 最终输出经过了 StrOutputParser 处理，所以可以直接输出流式输出 s
for s in chain.stream({"input": "全球经济"}):
    print(s, end="", flush=True)

尽管国际合作和协调在推动全球经济稳定和可持续发展方面可能会遇到一些困难和挑战，但我们仍然认为国际合作至关重要。通过共同努力克服障碍，各国可以共同应对全球经济面临的挑战，实现共同繁荣和可持续发展。国际合作不仅可以促进经济增长和贸易自由化，还能够解决全球性问题，为全球经济的发展开辟更广阔的前景。因此，我们仍然强调国际合作的重要性，并呼吁各国共同努力，共同应对全球经济挑战。

### Homework: 实现一个多链版本的代码生成，输入功能需求，输出2种（Python，Java）以上编程语言的代码实现。